## script to scale all histograms in coffea files
- saved in outputs/scale/
- also scales the cutflow dictionary
## adds the scaled files from multiple years.
- the histograms in the files must all have the same axes

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import mplhep as hep
hep.style.use("CMS")
from coffea import util
import itertools
import os, sys
import glob
import copy

sys.path.append('../python/')
from functions import *


### analysis categories

In [ ]:
label_map = getLabelMap()
label_to_int = {label: i for i, label in label_map.items()}
signal_cats = [ i for label, i in label_to_int.items() if '2t' in label]
pretag_cats = [ i for label, i in label_to_int.items() if 'pre' in label]
antitag_cats = [ i for label, i in label_to_int.items() if 'at' in label]



print('------ analysis category map --------')
for i, lab in label_map.items():
    print(f'{i}: {lab}')
print('-------------------------------------')


print('\n\n------ coffea file content --------')

for hname in loadCoffeaFile().keys():
    print(hname)
print('-------------------------------------')


## scale histograms in IOV

In [ ]:
coffeafiles = getCoffeaFilenames()

datasets = ['QCD', 'TTbar', 'JetHT', 'RSGluon']

IOV = '2016APV'
for ds in datasets:

    try:

        coffeafiles[ds]['unweighted'][IOV].keys()
        sections = coffeafiles[ds]['unweighted'][IOV].keys()

        files = []


        for s in sections:

            filename = coffeafiles[ds]['unweighted'][IOV][s]
            original_file = util.load(filename)

            file = copy.deepcopy(original_file)

            if 'JetHT' in ds:

                files.append(file)

            else:

                factor = toptag_sf**2 if 'TTbar' in ds else 1.0
                sf = lumi[IOV] * xs[ds][s] * factor / file['cutflow']['sumw']

                for key in file.keys():

                    if 'hist' in str(type(file[key])):
                        file[key] = file[key] * sf

                    elif 'cutflow' in key:
                        for cut in file[key].keys():
                            file[key][cut] = file[key][cut] * sf



                files.append(file)

                if 'RSGluon' in ds or 'ZPrime' in ds:

                    util.save(file, f'../outputs/scale/{ds}{s}_{IOV}.coffea')
                    print(f'saving ../outputs/scale/{ds}{s}_{IOV}.coffea')


        if 'RSGluon' not in ds and 'ZPrime' not in ds:

            file = files[0]

            for f in files[1:]:
                for key in file.keys():

                    if 'hist' in str(type(file[key])):
                        file[key] = file[key] + f[key]

                    elif 'cutflow' in key:
                        for cut in f[key].keys():
                            f[key][cut] = f[key][cut] + file[key][cut]

            util.save(file, f'../outputs/scale/{ds}_{IOV}.coffea')
            print(f'saving ../outputs/scale/{ds}_{IOV}.coffea')

    except:

        filename = coffeafiles[ds]['unweighted'][IOV]
        original_file = util.load(filename)
        file = copy.deepcopy(original_file)


        sf = lumi[IOV] * xs[ds] / file['cutflow']['sumw']

        for key in file.keys():

            if 'hist' in str(type(file[key])):
                file[key] = file[key] * sf

            elif 'cutflow' in key:

                for cut in file[key].keys():

                    file[key][cut] = file[key][cut] * sf


        util.save(file, f'../outputs/scale/{ds}_{IOV}.coffea')
        print(f'saving ../outputs/scale/{ds}_{IOV}.coffea')






## Combine JetHT files (blinded)


In [ ]:
IOVs = ['2016APV', '2016']#, '2017', '2018']

files = []
for IOV in IOVs:

    file = util.load(f'../outputs/scale/JetHT_{IOV}.coffea')
    files.append(file)
    
systs = 'nominal'#, 'pileupDown', 'pileupUp', 'prefiringDown', 'prefiringUp', 'pdfDown', 'pdfUp', 'btagDown', 'btagUp', 'jesDown', 'jesUp', 'jerDown', 'jerUp']
file = files[0]
for f in files[1:]:
    for key in file.keys():

        if 'hist' in str(type(file[key])):
            
            file[key] = file[key] + f[key]            

        elif 'cutflow' in key:
            for cut in f[key].keys():
                f[key][cut] = f[key][cut] + file[key][cut]  



# util.save(file, f'../outputs/scale/JetHT_blinded.coffea')
util.save(file, f'../outputs/scale/JetHT_2016all_unblinded.coffea')


## Combine MC files (for plotting)


In [ ]:
IOVs = ['2016APV', '2016', '2017', '2018']
datasets = ['QCD', 'TTbar', 'RSGluon']

files = []


# all all RSGluon samples
datasets += ['RSGluon'+str(int(b)) for b in np.linspace(1000,5000,9)]

for ds in datasets:

    for IOV in IOVs:

        file = util.load(f'../outputs/scale/{ds}_{IOV}.coffea')
        files.append(file)


    file = files[0]
    for f in files[1:]:
        for key in file.keys():

            if 'hist' in str(type(file[key])):
                file[key] = file[key] + f[key]

            elif 'cutflow' in key:
                for cut in f[key].keys():
                    f[key][cut] = f[key][cut] + file[key][cut]  



    util.save(file, f'../outputs/scale/{ds}_all.coffea')


## combine 2016noAPV and 2016APV

In [ ]:


IOVs = ['2016', '2016APV']
datasets = ['QCD', 'JetHT', 'TTbar']

# all all RSGluon samples
datasets += ['RSGluon'+str(int(b)) for b in np.linspace(1000,5000,9)]


for ds in datasets:
    
    files = []
    for IOV in IOVs:

        file = util.load(f'../outputs/scale/{ds}_{IOV}.coffea')
        files.append(file)


    file = files[0]
    for f in files[1:]:
        for key in file.keys():

            if 'hist' in str(type(file[key])):

                if 'systematic' in list(f[key].axes.name):
                    file[key] = file[key] + f[key][{'systematic':systs}]
                else:
                    file[key] = file[key] + f[key]

            elif 'cutflow' in key:
                for cut in f[key].keys():
                    f[key][cut] = f[key][cut] + file[key][cut]  



    util.save(file, f'../outputs/scale/{ds}_2016all.coffea')
    print(f'saving ../outputs/scale/{ds}_2016all.coffea')   
